In [6]:
import matplotlib.pyplot as plt
import cv2
import scipy.signal as signal
import numpy as np
import pandas as pd
import os

In [7]:
class Kinect(object):
    def __init__(self, kinect_data):
        self.raw_data = kinect_data

    def compute_num_timestamp(self):
        count_num = int(len(self.raw_data['time'])/25)

        return count_num

    def get_xyz_tolist(self):
        key_points_data = []
        for n in range(25):
            key_points_data.append([])
            for m in range(3):
                key_points_data[n].append([])

        for k in range(25):
            for m in range(self.compute_num_timestamp()):
                key_points_data[k][0].append(self.raw_data.iloc[m*25+k, 2])
                key_points_data[k][1].append(self.raw_data.iloc[m*25+k, 3])
                key_points_data[k][2].append(self.raw_data.iloc[m*25+k, 4])

        return key_points_data

    def get_timestamp(self):
        timestamp = []
        for t in range(self.compute_num_timestamp()):
            timestamp.append(self.raw_data.iloc[t*25, 6])

        return timestamp

    def get_maxtime(self):
        return max(self.get_timestamp())

In [8]:
class KinectSplit(Kinect):
    def __init__(self, kinect_data):
        Kinect.__init__(self, kinect_data)

    # 使用median5 和 Gaussian15，5对keypoint某一维数据过滤噪声
    def noise_reduction(self, oned_kdata):
        oned_kdata_mean = cv2.blur(np.array(oned_kdata), (1,5))
        oned_kdata_mean_gauss = cv2.GaussianBlur(oned_kdata_mean, (1,15), 5)
        oned_kdata_mean_gauss = np.squeeze(oned_kdata_mean_gauss)
        oned_kdata_mean_gauss_mean = cv2.blur(oned_kdata_mean_gauss, (1,5))

        return oned_kdata_mean_gauss_mean

    def sb_noise_reduction(self, oned_kdata):
        oned_kdata_mean = cv2.blur(np.array(oned_kdata), (1,5))
        oned_kdata_mean_gauss = cv2.GaussianBlur(oned_kdata_mean, (1,25), 10)
        oned_kdata_mean_gauss = np.squeeze(oned_kdata_mean_gauss)
        oned_kdata_mean_gauss_mean = cv2.blur(oned_kdata_mean_gauss, (1,5))
        return oned_kdata_mean_gauss_mean

    #  对spinebase数据滤波并返回一个数据列表
    def sb_z(self):
        sb_z_filt = self.sb_noise_reduction(self.get_xyz_tolist()[0][2])
        sb_z_filt = sb_z_filt.reshape(-1)
        sb_z_filt = sb_z_filt.tolist()

        return sb_z_filt

    #  对left&right elbow数据滤波并差分取绝对值，返回一个滤波后的列表
    def elbow_diff(self):
        elbow_left_filt = self.noise_reduction(self.get_xyz_tolist()[5][0]).reshape(-1).tolist()
        elbow_right_filt = self.noise_reduction(self.get_xyz_tolist()[9][0]).reshape(-1).tolist()
        elbow_diff = [abs(elbow_left_filt[i]-elbow_right_filt[i]) for i in range(len(elbow_left_filt))]
        elbow_diff = self.noise_reduction(elbow_diff).reshape(-1).tolist()

        return elbow_diff

    #  对left&right foot数据滤波并差分，返回一个滤波后的列表
    def foot_diff(self):
        foot_left_filt = self.noise_reduction(self.get_xyz_tolist()[15][2]).reshape(-1).tolist()
        foot_right_filt = self.noise_reduction(self.get_xyz_tolist()[19][2]).reshape(-1).tolist()
        foot_diff = [(foot_left_filt[i]-foot_right_filt[i]) for i in range(len(foot_left_filt))]
        foot_diff = self.noise_reduction(foot_diff).reshape(-1).tolist()

        return foot_diff

    #  获得行走阶段的起止点数据
    def get_walkingpoint(self):
        sb_z_filt = self.sb_z()

        sb_z_filt_diff = []
        for i in range(len(sb_z_filt)-1):
            sb_z_filt_diff.append(round((sb_z_filt[i+1]-sb_z_filt[i]), 5))

        walking_start_index = 0
        for i in sb_z_filt_diff:
            if i < -0.0050:
                walking_start_index = sb_z_filt_diff.index(i)
                break

        walking_end_index = 0
        for i in sb_z_filt_diff[-int(0.25*len(sb_z_filt_diff)):]:
            if i < 0.0050:
                walking_end_index = sb_z_filt_diff.index(i)
                break

        return walking_start_index, walking_end_index

    #  获得转身阶段的起止点数据
    def get_turningpoint(self):
        elbow_diff = self.elbow_diff()

        turning_start_index = 0
        for i in range(int(0.25*len(elbow_diff)),len(elbow_diff)-1):
            if (round((elbow_diff[i+1]-elbow_diff[i]), 5) < -0.0035) and (0.350 < elbow_diff[i] < 0.500):
                turning_start_index = i
                break

        turning_end_index = 0
        if turning_start_index+20 < len(elbow_diff): 
            for i in range(turning_start_index+20,len(elbow_diff)-1):
                if (elbow_diff[i+1]-elbow_diff[i]) < 0.0040 and (0.350 < elbow_diff[i] < 0.500):
                    turning_end_index = i
                    break

        return turning_start_index, turning_end_index

In [9]:
def get_file_path(dir_path):

    '''
    This function aims to get paths of all kinect data files
    and remove the paths of empty files.
    Need to import os and Kinect moudule.
    
    Parameters
    ----------
    dir_path: str
        the directory folder of files
    
    Return
    ------
    kinect_files_path: list
        a list of paths of kinect files
    '''

    #  get paths of Kinect files
    kinect_files_path = []
    for root, dirs, files in os.walk(dir_path):
        if root[-15:-11] == "3TUG":
            for file in os.listdir(root):
                if file == "modified_Kin.csv":
                    kinect_files_path.append(os.path.join(root, file))

    #  find paths of empty files
    empty_kpath = []
    for i in range(len(kinect_files_path)):
        kinect_data = Kinect(pd.read_csv(kinect_files_path[i]))
        if kinect_data.get_timestamp() == []:
            empty_kpath.append([i, kinect_files_path[i]])

    #  remove paths of empty files
    for i in range(len(empty_kpath)):
        kinect_files_path.remove(empty_kpath[i][1])

    return kinect_files_path

In [ ]:
def kin_clsbytime(kinect_files_path):
    '''
    This funtion classifies the kinect data files by their
    max timestamp.
    
    Parameters
    ----------
    kinect_files_path: list
        A list consists of paths of kinect files
    
    Return
    ------
    time_fift: list
        A list of kinect data files whose max timestamp
        less than fifteen
    time_fif2twenfive: list
        A list of kinect data files whose max timestamp 
        between fifteen and twenty five
    time_twenfive: list
        A list of kinect data files whose max timestamp
        more than twenty five
    '''
    #  将Kinect文件按完成时间分类
    time_fift = []
    time_fift2twenfive = []
    time_twenfive = []

    for i in range(len(kinect_files_path)):
        kinect_data = Kinect(pd.read_csv(kinect_files_path[i]))
        if kinect_data.get_maxtime() <= 15.0:
            time_fifteen.append(kinect_files_path[i])
        elif kinect_data.get_maxtime() <= 25.0:
            time_fifteen2twentyfive.append(kinect_files_path[i])
        else:
            time_twentyfive.append(kinect_files_path[i])
    
    return time_fift, time_fif2twenfive, time_twenfive

In [ ]:
def imu2kin_nrest_spltpts(kin_file_path, tps_dict):
    '''
    This function gets the nearest points of imu split time points
    in the data of kinect file.
    
    Parameters
    ----------
    kin_file_path: str
        The path of kinect data file.
    tps_dict: dict
        The dictionary producted from function kin_imutps_dict

    Return
    ------
    walking_start:int
        Index of walking start point in kinect data
    turning_start:int
        Index of turning start point in kinect data
    turning_end:int
        Index of turning end point in kinect data
    walking_end:int
        Index of walking end point in kinect data
    '''
    
    #  找到转身的两个点 划出原始数据的两个时间段，画图
    time_stamp = Kinect(pd.read_csv(kin_file_path)).get_timestamp()
    tps_df = tps_dict[kin_file_path][2]
    imu_walking_start = tps_df.iloc[0,0]
    imu_turning_start = tps_df.iloc[0,1]
    imu_turning_end = tps_df.iloc[0,2]
    imu_walking_end = tps_df.iloc[0,3]
    
    #  在Kinect数据点中寻找与IMU切分点最接近的点
    walking_start = find_nearest(time_stamp, imu_walking_start)
    turning_start = find_nearest(time_stamp, imu_turning_start)
    turning_end = find_nearest(time_stamp, imu_turning_end)
    walking_end = find_nearest(time_stamp, imu_walking_end)
    
    return walking_start, turning_start, turning_end, walking_end

In [10]:
def kin_imutps_dict(imu_file_path, kinect_files_path):
    '''
    This function returns a dictionary which contains path of kinect files 
    with their IMU walking and turning time points.
    Need to import pandas moudule.
    
    Parameters
    ----------
    imu_file_path: str
        the path of IMU data file
    kinect_files_path: array-like
        An array or array-like type contains paths of all kinect files

    Return
    ------
    path_imu_spltime: dict
        A dictionary whose keys are paths of kinect files and values are an array
        or array-like type of keys' test id, test date and IMU time points
    '''

    # 导入IMU数据的时间切割点
    imu_timepoints = pd.read_excel(imu_file_path)
    
    #  将文件路径按照社区名称分类,得到一个字典
    fil_clsfied_cen = {'Fortune Neighbourhood Elderly Centre':[],
                       'Shamshuipo Day Care Centre for the Elderly':[],
                       'Shun Lee Neighborhood Elderly Centre':[],
                       'Un Chau Neighborhood Elderly Centre':[]
                      }

    for path in kinect_files_path:
        for centre in fil_clsfied_cen.keys():
            if (centre in path):
                fil_clsfied_cen[centre].append(path)
    
    #  将IMU数据按照社区名称分类
    un_chau = imu_timepoints[imu_timepoints['centre'] == '元州']
    fortune = imu_timepoints[imu_timepoints['centre'] == '幸福']
    shamshuipo = imu_timepoints[imu_timepoints['centre'] == '深水埗']
    shun_lee = imu_timepoints[imu_timepoints['centre'] == '顺利']

    spltime_clsfied_cen = {'Fortune Neighbourhood Elderly Centre':fortune,
                           'Shamshuipo Day Care Centre for the Elderly':shamshuipo,
                           'Shun Lee Neighborhood Elderly Centre':shun_lee,
                           'Un Chau Neighborhood Elderly Centre':un_chau
                          }
    
    #  记录各个社区的测试时间
    test_date = {'Un Chau Neighborhood Elderly Centre':['2019-12-05', '2019-12-12'],
                 'Fortune Neighbourhood Elderly Centre':['2020-01-03', '2020-01-09'],
                 'Shamshuipo Day Care Centre for the Elderly':['2019-12-09', '2019-12-16'],
                 'Shun Lee Neighborhood Elderly Centre':['2019-11-05', '2019-12-20']}
    
    #  将IMU数据按照每个测试的文件路径对应存储
    path_imu_spltime = {}
    for centre_name in fil_clsfied_cen.keys():
        for file_path in fil_clsfied_cen[centre_name]:
            for _id in spltime_clsfied_cen[centre_name]['ID']:
                test_id = "test"+str(_id)
                if (test_id in file_path) and (test_date[centre_name][0] in file_path):
                    path_imu_spltime[file_path] = [test_id,
                                                   test_date[centre_name][0],
                                                   spltime_clsfied_cen[centre_name].loc[spltime_clsfied_cen[centre_name]['ID']==_id, 'first-1':'first-4']]
                elif (test_id in file_path) and (test_date[centre_name][1] in file_path):
                    path_imu_spltime[file_path] = [test_id,
                                                   test_date[centre_name][1],
                                                   spltime_clsfied_cen[centre_name].loc[spltime_clsfied_cen[centre_name]['ID']==_id, 'sec-1':'sec-4']]

    return path_imu_spltime

In [11]:
#  寻找数组中离某一给定数值最近的数 可以找到kin数据中的转身截断点
def find_nearest(array, value):
    '''
    This function gets an array (or array-like) and a value which needs to 
    be compared with elements in the array to find the nearest value in 
    the array, returns the index of the nearest value.
    Need to import numpy moudule
    
    Parameters
    ----------
    array: array or array-like
        An array which the nearest value is in 
    value: int or float (same as the elements in the parameter array)
        The value needs to be compared with the elements in the array
    
    Return
    ------
    idx: int
        The index of the nearest value in the array
    '''
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
def get_timefeature(kin_df):
    '''
    This function computes the features from time of 3MTUG test
    including percent of each stage (sitting, walking and turning)
    and turning time
    
    Parameter
    ---------
    kin_df: DataFrame
    DataFrame consists of time of each phase of 3MTUG test
    
    Example
    -------
        centre	ID	sec-1	sec-2	sec-3	sec-4	sec-end	sitting_1	sitting_2	walking_1	walking_2	turning_1	turning_time
    0	元州	1	3.065328	5.503728	8.143728	10.588528	15.858928	0.1933	0.3323	0.1538	0.1542	0.1665	2.64
    1	元州	2	6.763178	12.244778	15.284778	18.951978	22.193578	0.3047	0.1461	0.2470	0.1652	0.1370	3.04
    2	元州	4	2.551003	4.579803	6.608603	7.837403	11.485403	0.2221	0.3176	0.1766	0.1070	0.1766	2.03
    
    Return
    ------
    time_df: DataFrame
        The original DataFrame extended by kinect time features
    '''
    new_col_names = ['sitting_1','sitting_2','walking_1','walking_2','turning_1','turning_time']
    df_cp = kin_df.copy()
    col_names = df_cp.columns
    end_time = df_cp[col_names[-1]].copy()
    walking_start = df_cp[col_names[2]].copy()
    turning_start = df_cp[col_names[3]].copy()
    turning_end = df_cp[col_names[4]].copy()
    walking_end = df_cp[col_names[5]].copy()
    
    time_df[new_col_names[0]] = round(walking_start/end_time, 4)
    time_df[new_col_names[1]] = round((end_time-walking_end)/end_time, 4)
    time_df[new_col_names[2]] = round((turning_start-walking_start)/end_time, 4)
    time_df[new_col_names[3]] = round((walking_end-turning_end)/end_time, 4)
    time_df[new_col_names[4]] = round((turning_end-turning_start)/end_time, 4)
    time_df[new_col_names[5]] = round(turning_end-turning_start, 2)
    
    return time_df

In [13]:
#def kin_pic(kinect_files_path, pic_num, spltpts=False, sub=False, save_path):
'''
    This function draws pictures of specific kinect keypoints data
    including spinebase, left&right elbows and left&right ankles
    Need to import kinect, matplotlib moudule
    
    Parameters
    ----------
    kinect_files_path: array or array-like
        A list of kinect data files
    pic_num: int
        The number of pics in the files list to draw
    spltpts: Boolean, Default=False
        Whether getting the split points in the data such as turning 
        point, walking_start point and drawing the vertical lines in
        the data pics
    sub: Boolean, Default=False
        Whether drawing pics in a subplot
    save_path: str
        The directory path to save the pics
    
    Return
    ------
    None. Show the specific kinect data pics
'''   

'\n    This function draws pictures of specific kinect keypoints data\n    including spinebase, left&right elbows and left&right ankles\n    Need to import kinect, matplotlib moudule\n    \n    Parameters:\n    kinect_files_path: array or array-like\n    A list of kinect data files\n    pic_num: int\n    The number of pics in the files list to draw\n    spltpts: Boolean, Default=False\n    Whether getting the split points in the data such as turning \n    point, walking_start point and drawing the vertical lines in\n    the data pics\n    sub: Boolean, Default=False\n    Whether drawing pics in a subplot\n    save_path: str\n    The directory path to save the pics\n    \n    Return: None\n    Show the specific kinect data pics\n'